#### Zadanie:

Identifikujte sedenia na základe kvartilového odhadu časového okna (STT_Q). (= hornyQ + 1.5*Qrozpatie)


Odovzdajte zdrojový kód programu + logovací súbor doplnený o identifikáciu používateľov (v jednom zip).

Príkazy v Pythone vhodné pre riešenie tohto zadania:

`import pandas as pd
pd.read_csv() - načítanie log súboru
quantile() - metóda na vypočítanie kvartilu (0.25 - dolný kvartil, 0.75 - horný kvartil)
`

In [1]:
import pandas as pd
import numpy as np

In [2]:
# opening the file we will be working with and assigning it to the variabel file for latter use
file = pd.read_csv("10_Identifikácia_sedení_STT_MEAN.csv",  encoding="utf-8",  escapechar=";", index_col=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# calculating the STT_Q = hornyQ + 1.5*(HornyQ-DolnyQ)

STT_Quart = file["Length"].quantile(.75) + 1.5*(file["Length"].quantile(.75) - file["Length"].quantile(.25))

In [4]:
STT_Quart

12.5

In [5]:
# initializing the empty array for STT_Q id numbers
# initializing the temp var STT_Q_id to store temp ID numbers of the STT_Q position index
STT_Q = []
STT_Q_id = 0

# temp variables for comparison
previous_seq_id = 0
previous_length = 0
# we increment when seq_id != previous_seq_id

# we increment when seq_id == previous_seq_id and:
#     1. previous_length is NaN and length variable is bigger than STT_Quart value
#     2. previous_length is NaN and length is NaN
#     3. previous_length is more than STT_Quart 
#     4. previous_length is less than STT_Quart and length is NaN

for seq_id, length in zip(file["SEQUENCE_ID"], file["Length"]):
    if ((previous_seq_id != seq_id)
    or ((previous_seq_id == seq_id) and (np.isnan(previous_length) and length > STT_Quart))
    or ((previous_seq_id == seq_id) and (np.isnan(previous_length) and np.isnan(length)))
    or ((previous_seq_id == seq_id) and (previous_length > STT_Quart))
    or ((previous_seq_id == seq_id) and (np.isnan(previous_length) and length < STT_Quart))):
    
        STT_Q_id += 1
        STT_Q.append(STT_Q_id)
        previous_seq_id = seq_id
        previous_length = length

    else:
        STT_Q.append(STT_Q_id)
        previous_seq_id = seq_id
        previous_length = length

In [6]:
file.insert(10, 'STT_Q', STT_Q)

In [7]:
# save the file
file.to_csv("11_Identifikácia_sedení_STT_Q.csv", encoding="utf-8",  escapechar=";", index=False)